In [6]:
import requests
import time
import concurrent.futures
import base64

In [7]:
def base64_encode(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def test_single_request(encoded_image, url):
    start_time = time.time()
    response = requests.post(url, json={"image": encoded_image})
    elapsed_time = time.time() - start_time

    if response.status_code == 200:
        return elapsed_time
    else:
        return None

In [8]:
def load_test(encoded_image, url, num_requests):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(test_single_request, encoded_image, url) for _ in range(num_requests)]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]

    results = [result for result in results if result is not None]
    return results

In [14]:
def run_load_tests(url, sample_image, num_requests):

    encoded_image = base64_encode(sample_image)
    
    print(f"Starting load test with {num_requests} concurrent requests...")
    latencies = load_test(encoded_image, url, num_requests)

    average_latency = sum(latencies) / len(latencies)
    print(f"Average latency: {average_latency:.4f} seconds")
    print(f"Min latency: {min(latencies):.4f} seconds")
    print(f"Max latency: {max(latencies):.4f} seconds")

run_load_tests("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 1)

Starting load test with 1 concurrent requests...
Average latency: 0.5274 seconds
Min latency: 0.5274 seconds
Max latency: 0.5274 seconds


### Multi requests ###

In [15]:
def test_batch_request(encoded_images, url):
    start_time = time.time()
    response = requests.post(url, json={"images": encoded_images})
    elapsed_time = time.time() - start_time

    if response.status_code == 200:
        return elapsed_time, response.json()
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None

In [32]:
def concurrent_batch_test(encoded_images, url, num_requests):
   
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(test_batch_request, encoded_images, url) for _ in range(num_requests)]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]
    
    results = [result for result in results if result is not None]
    return results

In [44]:
def run_load_test_batch(url, sample_image, batch_size, num_requests):

    encoded_image = base64_encode(sample_image)
    encoded_images = [encoded_image] * batch_size

    print(f"Starting batch test with {num_requests} concurrent requests...")
    test_results = concurrent_batch_test(encoded_images, url, num_requests)

    latencies = [result[0] for result in test_results]
    average_latency = sum(latencies) / len(latencies)
    print(f"Average latency: {average_latency:.4f} seconds")
    print(f"Min latency: {min(latencies):.4f} seconds")
    print(f"Max latency: {max(latencies):.4f} seconds")

# local
run_load_test_batch("http://localhost:9000/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 5.8854 seconds
Min latency: 1.3917 seconds
Max latency: 8.4781 seconds


In [38]:
# Enclave without KMS test 1
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 36.7901 seconds


In [40]:
# Public test 1
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 12.6687 seconds


In [43]:
# Enclave with KMS test 1
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 16.0052 seconds


In [47]:
# Enclave without KMS test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 7.9189 seconds
Min latency: 4.6721 seconds
Max latency: 10.5994 seconds


In [49]:
# Public test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 7.7820 seconds
Min latency: 3.0985 seconds
Max latency: 10.1374 seconds


In [46]:
# Enclave with KMS test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 10, 50)

Starting batch test with 50 concurrent requests...
Average latency: 8.1429 seconds
Min latency: 4.3737 seconds
Max latency: 11.7411 seconds


### Larger batch size ###

In [52]:
# Enclave without KMS test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 20, 50)

Starting batch test with 50 concurrent requests...
Average latency: 16.4848 seconds
Min latency: 8.9849 seconds
Max latency: 25.3730 seconds


In [50]:
# Public test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 20, 50)

Starting batch test with 50 concurrent requests...
Average latency: 20.1880 seconds
Min latency: 9.0492 seconds
Max latency: 26.8044 seconds


In [51]:
# Enclave with KMS test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 20, 50)

Starting batch test with 50 concurrent requests...
Average latency: 20.9921 seconds
Min latency: 9.0141 seconds
Max latency: 26.1185 seconds


### More concurrent requests ###

In [54]:
# Enclave without KMS test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 20, 100)

Starting batch test with 100 concurrent requests...
Average latency: 17.4543 seconds
Min latency: 8.8289 seconds
Max latency: 25.4990 seconds


In [53]:
# Public test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 20, 100)

Starting batch test with 100 concurrent requests...
Average latency: 16.4251 seconds
Min latency: 9.2622 seconds
Max latency: 22.8421 seconds


In [55]:
# Enclave with KMS test 2
run_load_test_batch("http://localhost:8888/predict/", "public/sample_images/sample_cat.jpeg", 20, 100)

Starting batch test with 100 concurrent requests...
Average latency: 18.3277 seconds
Min latency: 7.4616 seconds
Max latency: 25.9433 seconds
